# <B> Checker for deploy fucntions </B>
* Container: codna_pytorch_p39

### This is to check deploy functions in inference.py

### Check functions in inference.py with local env.
[중요] inference.py를 만들어 주어야 함
* model_fn: 학습한 모델 로드
* input_fn: endpoint invocation시 전달 되는 input 처리 하는 함수
* predict_fn: forword propagation, input_fn의 이후 호출 
* output_fn: 유저에게 결과 전달

- 사용자 정의 inference 코드를 정의해서 사용하기 전에, 노트북에서 사전 테스트 및 디버깅을 하고 진행하면 빠르게 추론 개발을 할수 있습니다.
- 디폴트 inference code (input_fn, model_fn, predict_fn, output_fn) 을 사용해도 되지만, 상황에 따라서는 사용자 정의가 필요할 수 있습니다. 디폴트 코드는 아래 링크를 참고 하세요.
    - [Deploy PyTorch Models](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#deploy-pytorch-models)
    - [디폴트 inference Code](https://github.com/aws/sagemaker-pytorch-inference-toolkit/blob/master/src/sagemaker_pytorch_serving_container/default_pytorch_inference_handler.py)
    
 ### 로컬 모드 수행시, 새로운 로컬모드 수행을 위해서는 이전 사용했던 도커는 반드시 stop 해줘야 한다
* docker ps -a 로 현재 수행중인 contatiner ID 확인 후
* docker stop "<<contatiner ID>>"
* docker container prune -f

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## function checker

* 학습된 모델 경로 이동 (S3 -> Local)

In [2]:
import os

In [3]:
artifact_path = "s3://sm-anomaly-detection/ad-ts/training/model-output/ad-ts-experiments-0428-02541682650451/model.tar.gz"
model_data_dir = "./model/"
os.makedirs(model_data_dir, exist_ok=True)

In [4]:
%%sh -s {artifact_path} {model_data_dir}

artifact_path=$1
model_data_dir=$2

echo $artifact_path
echo $model_data_dir

# 기존 데이터 삭제
rm -rf $model_data_dir/*

# 모델을 S3에서 로컬로 다운로드
aws s3 cp $artifact_path $model_data_dir

# 모델 다운로드 폴더로 이동
cd $model_data_dir

# 압축 해제
tar -xvf model.tar.gz 

s3://sm-anomaly-detection/ad-ts/training/model-output/ad-ts-experiments-0428-02541682650451/model.tar.gz
./model/
download: s3://sm-anomaly-detection/ad-ts/training/model-output/ad-ts-experiments-0428-02541682650451/model.tar.gz to model/model.tar.gz
best_model.pth


* check for "model_fn"

In [5]:
from src.deploy.inference import model_fn

In [6]:
model = model_fn(model_data_dir)

Input dim: 20, from num_features(4), shingle_size(4) and emb_size(4)


* check for "input_fn"

    * download preprocessed data to local

In [7]:
output_path = "s3://sm-anomaly-detection/ad-ts/preprocessing/output/"
!aws s3 sync $output_path ./data/preprocessing --quiet

In [8]:
import json
import pickle
from src.deploy.inference import input_fn

In [9]:
def from_pickle(obj_path):
    with open(file=obj_path, mode="rb") as f:
        obj=pickle.load(f)
    return obj

In [10]:
data_path = "./data/preprocessing/data_x_scaled_shingle.pkl"
data = from_pickle(data_path)
data = data[0, :]
dtype = data.dtype
shape = data.shape

In [11]:
print (f'data: {data}')
print (f'dtype: {dtype}')
print (f'shape: {shape}')

data: [ 0.         -1.2944116  -0.11915769 -1.2944116  -0.51090899 -0.88238043
 -0.97225508 -0.88238043 -0.70263113 -0.92357365 -1.00811606 -0.92357365
 -0.75448883 -0.47232556 -0.47232556 -0.47232556 -0.47232556]
dtype: float64
shape: (17,)


In [12]:
payload={"INPUT": data.tolist(), "SHAPE":shape, "DTYPE": str(dtype)}
payload = json.dumps(payload)

In [13]:
input_data = input_fn(request_body=payload, request_content_type="application/json")
input_data

request_body: string


tensor([[ 0.0000, -1.2944, -0.1192, -1.2944, -0.5109, -0.8824, -0.9723, -0.8824,
         -0.7026, -0.9236, -1.0081, -0.9236, -0.7545, -0.4723, -0.4723, -0.4723,
         -0.4723]], device='cuda:0')

* check for "predict_fn"

In [28]:
from src.deploy.inference import predict_fn

In [29]:
pred = predict_fn(input_data, model)

In [30]:
pred

[{'ANOMALY_SCORE_SAP': 0.3146185,
  'URLS_ATTRIBUTION_SCORE': 0.035414316,
  'USERS_ATTRIBUTION_SCORE': 0.026715353,
  'CLICKS_ATTRIBUTION_SCORE': 0.02899459,
  'RESIDUALS_ATTRIBUTION_SCORE': 0.03928221,
  'ANOMALY_SCORE': 0.1304064691066742}]

* check for "output_fn"

In [31]:
from src.deploy.inference import output_fn

In [32]:
output_fn(pred)

'{"pred": [{"ANOMALY_SCORE_SAP": 0.3146184980869293, "URLS_ATTRIBUTION_SCORE": 0.03541431576013565, "USERS_ATTRIBUTION_SCORE": 0.02671535313129425, "CLICKS_ATTRIBUTION_SCORE": 0.028994590044021606, "RESIDUALS_ATTRIBUTION_SCORE": 0.03928221017122269, "ANOMALY_SCORE": 0.1304064691066742}]}'

## Deploy

* params for deployment

In [121]:
import time
import sagemaker

In [122]:
# Set to True to enable SageMaker to run locally
local_mode = True

if local_mode:
    instance_type = "local_gpu"
    
    from sagemaker.local import LocalSession
    import os
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    
else:
    instance_type = "ml.p3.2xlarge" #"ml.p3.2xlarge"#"ml.g4dn.8xlarge"#"ml.p3.2xlarge", 'ml.p3.16xlarge' , ml.g4dn.8xlarge
    sagemaker_session = sagemaker.Session()
    

prefix = "checker"
strEndpointName = f"endpoint-{prefix}-{int(time.time())}"
    
print (f'instance_type: {instance_type}')
print (f'sagemaker_session: {sagemaker_session}')
print (f"Endpoint-name: {strEndpointName}")

instance_type: local_gpu
sagemaker_session: <sagemaker.local.local_session.LocalSession object at 0x7f0be9892730>
Endpoint-name: endpoint-checker-1680163338


* deployment

In [123]:
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.serializers import CSVSerializer, NumpySerializer, JSONSerializer
from sagemaker.deserializers import JSONDeserializer, NumpyDeserializer

In [ ]:
local_esimator = PyTorchModel(
    source_dir="./source/serve",
    entry_point="inference.py",
    model_data=pm.get_params(key=strPrefix + "S3-MODEL-ARTIFACT"),
    role=pm.get_params(key=strPrefix + "SAGEMAKER-ROLE-ARN"),
    framework_version='1.12.1',
    py_version='py38',
    model_server_workers=1,
)